In [ ]:
import re
from datetime import datetime

import benchlingapi
import Bio.Restriction as Restriction
import pandas as pd
import pygsheets
import toml
from Bio.Seq import Seq
from tqdm.auto import tqdm

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.cloning.commands.parser as cmd_parser
import paulssonlab.cloning.commands.semantics as cmd_semantics
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.util as cloning_util
import paulssonlab.cloning.workflow as workflow
from paulssonlab.api.util import base_url

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"])

# Data

# Design

In [ ]:
def _format_seq(seq):
    # TODO: mixed bases in upper case for IDT
    return str(sequence.get_seq(seq)).lower()

In [ ]:
def overhangs_for(x):
    return (x["Upstream overhang"], x["Downstream overhang"])

In [ ]:
part_types = reg[("LIB", "parts", "Part types")]

In [ ]:
part_types["5UTR_BC"]

In [ ]:
part_types["5UTR_1"]

In [ ]:
part_types["5UTR_2"]

In [ ]:
ribozymes_tsv = """
Name	Sequence
RiboJ	AGCTGTC|ACCGGATGTGCTTTCCGGTCTGATGAGTCCGTGAGGACGAAACAGCCTCTACAAATAATTTTGTTTAA
AraJ	AGTGGTC|GTGATCTGAAACTCGATCACCTGATGAGCTCAAGGCAGAGCGAAACCACCTCTACAAATAATTTTGTTTAA
BydvJ	AGGGTGTC|TCAAGGTGCGTACCTTGACTGATGAGTCCGAAAGGACGAAACACCCCTCTACAAATAATTTTGTTTAA
CchJ	AGTTCCAGTC|GAGACCTGAAGTGGGTTTCCTGATGAGGCTGTGGAGAGAGCGAAAGCTTTACTCCCGCACAAGCCGAAACTGGAACCTCTACAAATAATTTTGTTTAA
ElvJ	AGCCCCATA|GGGTGGTGTGTACCACCCCTGATGAGTCCAAAAGGACGAAATGGGGCCTCTACAAATAATTTTGTTTAA
LtsvJ	AGTACGTC|TGAGCGTGATACCCGCTCACTGAAGATGGCCCGGTAGGGCCGAAACGTACCTCTACAAATAATTTTGTTTAA
PlmJ	AGTCATAAGTC|TGGGCTAAGCCCACTGATGAGTCGCTGAAATGCGACGAAACTTATGACCTCTACAAATAATTTTGTTTAA
SarJ	AGACTGTC|GCCGGATGTGTATCCGACCTGACGATGGCCCAAAAGGGCCGAAACAGTCCTCTACAAATAATTTTGTTTAA
ScmJ	AGCGCTGTC|TGTACTTGTATCAGTACACTGACGAGTCCCTAAAGGACGAAACACCGCCTCTACAAATAATTTTGTTTAA
RiboJ10	AGCGCTC|AACGGGTGTGCTTCCCGTTCTGATGAGTCCGTGAGGACGAAAGCGCCTCTACAAATAATTTTGTTTAA
RiboJ51	AGTAGTC|ACCGGCTGTGCTTGCCGGTCTGATGAGCCTGTGAAGGCGAAACTACCTCTACAAATAATTTTGTTTAA
RiboJ53	AGCGGTC|AACGCATGTGCTTTGCGTTCTGATGAGACAGTGATGTCGAAACCGCCTCTACAAATAATTTTGTTTAA
RiboJ54	AGGGGTC|AGTTGATGTGCTTTCAACTCTGATGAGTCAGTGATGACGAAACCCCCTCTACAAATAATTTTGTTTAA
RiboJ57	AGAAGTC|AATTAATGTGCTTTTAATTCTGATGAGTCGGTGACGACGAAACTTCCTCTACAAATAATTTTGTTTAA
RiboJ60	AGTCGTC|AAGTGCTGTGCTTGCACTTCTGATGAGGCAGTGATGCCGAAACGACCTCTACAAATAATTTTGTTTAA
RiboJ64	AGGAGTC|AATTAATGTGCTTTTAATTCTGATGAGACGGTGACGTCGAAACTCCCTCTACAAATAATTTTGTTTAA
"""

import io

ribozymes = pd.read_csv(io.StringIO(ribozymes_tsv), sep="\s+", index_col=0)
ribozymes["Cleaved"] = ribozymes["Sequence"].map(lambda x: x.split("|")[1])
ribozymes["Sequence"] = ribozymes["Sequence"].str.replace("|", "", regex=False)

In [ ]:
workflow.add_overhangs(riboj, overhangs_for(part_types["5UTR_1"]))

In [ ]:
(
    ribozymes.loc["RiboJ"]["Cleaved"]
    + part_types["5UTR_1"]["Downstream overhang"]
    + 35 * "N"
    + "a"
)

In [ ]:
sigw = reg[("LIB", "parts")]["sigW"]["Sequence"]

In [ ]:
sigw[1:]

# Weiss

In [ ]:
len(reg[("LIB", "parts")]["B0032m_BC"]["Sequence"])

In [ ]:
reg[("LIB", "parts")]["B0032m_BC"]["Sequence"] + sigw[4:]

In [ ]:
reg[("LIB", "parts")]["BCD22"]["Sequence"] + sigw[4:]

In [ ]:
reg[("LIB", "parts")]["BCD14"]["Sequence"] + sigw[4:]

In [ ]:
reg[("LIB", "parts")]["BCD14"]["Sequence"]  # + sigw[4:]

# Twist flank primers

In [ ]:
olib = reg[("oLIB", "oligos")]
lib_parts = reg[("LIB", "parts")]
olt = reg[("oLT", "oligos")]

In [ ]:
primer_binding = (
    lib_parts["Eaton_library_prefix"]["Sequence"],
    lib_parts["Eaton_library_suffix"]["Sequence"],
)

In [ ]:
primer_binding

In [ ]:
sequence.reverse_complement(primer_binding[1])

# RBS library order

In [ ]:
seqs = {
    "SigW_RBSlib1_weak": "ACCGGATGTGCTTTCCGGTCTGATGAGTCCGTGAGGACGAAACAGCCTCTACAAATAATTTTGTTTAACCATKRWGGCCAYKGSGGACACAKA",
    "SigW_RBSlib1_strong": "ACCGGATGTGCTTTCCGGTCTGATGAGTCCGTGAGGACGAAACAGCCTCTACAAATAATTTTGTTTAACCATAACTSASRKARGCGTTMTYTA",
}

In [ ]:
cleave5 = len(
    ribozymes.loc["RiboJ"]["Cleaved"] + part_types["5UTR_1"]["Downstream overhang"]
)
cleave3 = 1
seqs = {name: seq[cleave5:-cleave3] for name, seq in seqs.items()}

In [ ]:
overhangs = overhangs_for(part_types["5UTR_2"])
random_bases = (
    "GCTTCA",
    "TGCTAA",
)  # to add between BsmBI recognition site and ends of oligos
flanks = ("CGTCTCGGTCTCa", "tGAGACCgGAGACG")  # storage vector BsmBI flanks
seqs_to_order = {}
for name, seq in seqs.items():
    seqs_to_order[name] = workflow.add_flanks(
        workflow.add_overhangs(seq.lower(), overhangs),
        # [flanks, random_bases, primer_binding],
        [flanks, primer_binding],
    )

In [ ]:
date = datetime.now().strftime("%-m/%-d/%Y")

description = "Added Eaton flanks, to be amplified with oLIB45/46."

base_row = {
    "Author": "Jacob Quinn Shenker",
    "Vendor": "IDT",
    "Type": "Primer",
    "Date": date,
    "Order date": date,
    "Description": description,
}

rows = []

for seq_name, seq in seqs_to_order.items():
    name = f"{seq_name}_v3"
    row = {
        "Name": name,
        "Sequence": str(seq).upper(),
        # "Description": description,
        **base_row,
    }
    rows.append(row)

In [ ]:
[len(row["Sequence"]) for row in rows]

In [ ]:
for row in rows:
    olt[olt.next_id()] = row

In [ ]:
olt.save()

In [ ]:
olt.clear_cache()

In [ ]:
rows

# Ribozyme order

In [ ]:
import random


def random_bases(n):
    return "".join(random.choice("ATCG") for _ in range(max(n, 0)))

In [ ]:
seqs = ribozymes["Sequence"].to_dict()

In [ ]:
min_length = 125  # for gblock
overhangs = overhangs_for(part_types["5UTR_1"])
random_flanks = (
    "GCTTCA",
    "TGCTAA",
)  # to add between BsmBI recognition site and ends of oligos
flanks = ("CGTCTCGGTCTCa", "tGAGACCgGAGACG")  # storage vector BsmBI flanks
seqs_to_order = {}
for name, seq in seqs.items():
    seq = workflow.add_flanks(
        workflow.add_overhangs(seq.lower(), overhangs),
        [flanks, random_flanks],
    )
    seq += random_bases(min_length - len(seq))
    seqs_to_order[name] = seq

In [ ]:
prefix = "oLT"
id_num = 52
for seq_name, seq in seqs_to_order.items():
    for sense in (True,):
        if sense:
            oligo_seq = seq
        else:
            oligo_seq = sequence.reverse_complement(seq)
        id_ = f"{prefix}{id_num}"
        # name = f"{seq_name}_{'sense' if sense else 'antisense'}"
        print(f"{id_}\t{seq_name}\t{_format_seq(oligo_seq)}")
        # print(f"{id_}\t{_format_seq(oligo_seq)}")
        # print(f"{name}\t{len(oligo_seq)}")
        id_num += 1